In [1]:
import os, sys
import reaxff_convert as rc
import pandas as pd
import numpy as np

from reaxFF_data import reaxFF_data, bidict

# Automated Parameter Collation

The previous attempt was slightly disappointing. The energetics are different, but direct overlapping of parameters are (understably) not suitable. New methods needs to be sought. Dropping CO2 parameters as it seems that it is not very necessary. /ref Goddard, William

## Parameters Subsets
- Fe/O/Water: ffield.reax.Fe_O_C_H
- Water: ffield.reax.O_H
- Fe/C/H: ffield.reax.Fe_C_H
- CO2: ffield.reax.P_Na_Si_S_N_O_H_C

In [2]:
species = ['C', 'H', 'O', 'Fe']
potential_fd = os.path.join(os.getcwd(), "potentials")

water_fd = os.path.join(potential_fd, "ffield.reax.O_H")
fe_o_water_fd = os.path.join(potential_fd, "ffield.reax.Fe_O_C_H")
fe_c_h_fd = os.path.join(potential_fd, "ffield.reax.Fe_C_H")
fe_o_c_fd = os.path.join(potential_fd, "ffield.reax.Fe_O_C")

## Extracting Relevant Parameters

In [3]:
water_data = reaxFF_data(['H', 'O'])
water_data.read_lammps(water_fd)
water_data.clean_params()

fe_o_water_data = reaxFF_data(species)
fe_o_water_data.read_lammps(fe_o_water_fd)
fe_o_water_data.clean_params()

fe_c_h_data = reaxFF_data(['H', 'C', 'Fe'])
fe_c_h_data.read_lammps(fe_c_h_fd)
fe_c_h_data.clean_params()

fe_o_c_data = reaxFF_data(['C', 'O', 'Fe'])
fe_o_c_data.read_lammps(fe_o_c_fd)
fe_o_c_data.clean_params()

## Merging Parameters 2nd Try

In order, the sequence to be merged should be:

1. fe_o_water_data for the broadest coverage of useful activity
    - Is based on original Fe/Fe and O-H bulk water description. Only trained new Fe-O. Tuned with Fe-OH reactions
2. fe_c_h_data for the secondary coverage of important fe reactions
    - Retained old Fe/Fe and Fe/H data, and retrained various Fe/C data
3. water_data for the final good water interactions

In [4]:
master_data = reaxFF_data(species)
for i,k in master_data.params.items():
    k['source'] = float('nan')
steps = [master_data.copy()]

In [5]:
# First make the new df
steps.append(steps[-1].copy())

# # Combine data
for i, k in steps[-1].params.items():
    if i == 'general':
        k.update(fe_o_water_data.params[i])
        k['source'] = 'fe_o_water'
    else:
        cur_param_df = fe_o_water_data.params[i][k.columns.difference(['source'])]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'fe_o_water'
        k.update(changes_df)

# Merging Parameters with new Fe/C/O

In [7]:
new_steps = [steps[-1].copy()]

In [38]:
# First make the new df
new_steps.append(new_steps[-1].copy())

# # Combine data
for i, k in new_steps[-1].params.items():
    if i not in ['bonds', 'off_diagonal', 'angles']:
        continue
    else:
        cur_param_df = fe_o_c_data.params[i][k.columns.difference(['source'])]
        cur_param_df = cur_param_df[[('C' in x and 'Fe' in x) for x in cur_param_df.index]]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'fe_o_c'
        print(changes_df)
        k.update(changes_df)

           Edis1 LPpen n.u.1    pbe1 pbo5 13corr pbo6   kov    pbe2  pbo3  \
symbols                                                                     
C-Fe     78.6009     0     0  0.8712 -0.3      1   36  0.01  1.1811 -0.35   

        pbo4 Etrip    pbo1   pbo2 ovcorr n.u.2  source  
symbols                                                 
C-Fe      15     1 -0.1666  2.611      1     0  fe_o_c  
          Ediss      Ro  gamma  rsigma rpi rpi2   source
symbols                                                 
C-Fe     0.5077  0.3058  6.291  1.2466  -1    -1  fe_o_c
         Theta,o       ka       kb pv1     pv2 kpenal     pv3  source
symbols                                                              
C-C-Fe     7.031  17.9932   0.4327   0  0.1623      0   0.869  fe_o_c
C-O-Fe   99.3914   1.9671  10.5677   0  0.3631      0  0.0002  fe_o_c
C-Fe-C    2.3514   1.5811     0.45   0  0.3458      0  2.7984  fe_o_c
C-Fe-Fe  55.5054    0.599   3.6398   0  0.9539      0  0.7441  fe_o_c
O-C-Fe

# Try a mix _a la_ hybrid functionals

## 50/50 fe_o_water and fe_o_c

In [6]:
 fe_o_water_fe_o_c_5050 = steps[-1].copy()

# # Combine data
for i, k in fe_o_water_fe_o_c_5050.params.items():
    if i == 'general':
        continue
    else:
        new_param_df = fe_o_c_data.params[i][k.columns.difference(['source'])]
        changes_df = (k.loc[new_param_df.index] + new_param_df)/2
        changes_df['source'] = '0.5fe_o_c0.5fe_o_water'
        k.update(changes_df)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  if __name__ == '__main__':


## 50/50 fe_o_water and fe_o_c with water overwrite

In [8]:
 fe_o_water_fe_o_c_5050_water = fe_o_water_fe_o_c_5050.copy()

# # Combine data
for i, k in fe_o_water_fe_o_c_5050_water.params.items():
    if i == 'general':
        continue
    else:
        cur_param_df = water_data.params[i][k.columns.difference(['source'])]
        changes_df = k.reset_index().merge(cur_param_df.reset_index(),how="right").set_index('symbols')
        changes_df['source'] = 'water'
        k.update(changes_df)

# Writing back to File

In [32]:
for i, step in enumerate(steps):
    step.clean_params()
    step.gen_species2id()
    
    new_FF = step.write_lammps()
    
    with open(os.path.join("test_potentials", "new_potential_step_{}.reax".format(i)), 'w') as f:
        f.write(new_FF)

In [9]:
# 5050 potentials
for i, j in zip([fe_o_water_fe_o_c_5050_water, fe_o_water_fe_o_c_5050],
                ['fe_o_water_fe_o
                 _c_5050_water', 'fe_o_water_fe_o_c_5050']):
    i.description = j
    i.clean_params()
    i.gen_species2id()
    i_str = i.write_lammps()
    
    with open(os.path.join("5050_potentials", j+".reax"), 'w') as f:
        f.write(i_str)

In [33]:
import datetime
currentDT = datetime.datetime.now()

fe_o_water_and_fe_o_c = new_steps[-1].copy()
fe_o_water_and_fe_o_c.description = "Combined fe_o_water_and_fe_o_c " + str(currentDT)
fe_o_water_and_fe_o_c.clean_params()
fe_o_water_and_fe_o_c.gen_species2id()
fe_o_water_and_fe_o_c_str = fe_o_water_and_fe_o_c.write_lammps()

with open(os.path.join("fe_o_c_potentials", "bonds_od_angles_c_fe.reax"), 'w') as f:
    f.write(fe_o_water_and_fe_o_c_str)

In [8]:
fe_o_water_and_fe_c_h = new_steps[-1].copy()
fe_o_water_and_fe_c_h.description = "Combined fe_o_water_and_fe_c_h"
fe_o_water_and_fe_c_h.clean_params()
fe_o_water_and_fe_c_h.gen_species2id()
fe_o_water_and_fe_c_h_str = fe_o_water_and_fe_c_h.write_lammps()

with open(os.path.join("test_potentials", "fe_o_water_and_fe_c_h.reax"), 'w') as f:
    f.write(fe_o_water_and_fe_c_h_str)

In [ ]:
new_FF_name = ['water', 'fe_o_water', 'fe_c_h', 'fe_o_c']
old_FF = [water_data, fe_o_water_data, fe_c_h_data, fe_o_c_data]

for FF_data, FF_name in zip(old_FF, old_FF_name): 
    outstr = FF_data.write_lammps()
    
    with open(os.path.join("test_potentials", "{}.reax".format(FF_name)), 'w') as f:
        f.write(outstr)

In [4]:
old_FF_name = ['water', 'fe_o_water', 'fe_c_h', 'fe_o_c']
old_FF = [water_data, fe_o_water_data, fe_c_h_data, fe_o_c_data]

for FF_data, FF_name in zip(old_FF, old_FF_name): 
    outstr = FF_data.write_lammps()
    
    with open(os.path.join("test_potentials", "{}.reax".format(FF_name)), 'w') as f:
        f.write(outstr)